Задание 2. Итерационные методы решения СЛАУ (метод простой итерации, метод Зейделя).

In [1]:
import numpy as np
import pandas as pd
from typing import Callable
from enum import Enum
from numpy import linalg as LA
import math

In [2]:
class SolvingMethod(Enum):
    SIMPLE_ITERATION = 1,
    SEIDEL = 2


def convert_system(A: np.ndarray, b: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    inv_diag = LA.inv(np.diag(np.diag(A)))
    return np.identity(A.shape[0]) - (inv_diag @ A), inv_diag @ b


def iterate(H: np.ndarray, g: np.ndarray, stop: Callable[[float, float], bool], limit: int) -> tuple[np.ndarray, int]:    
    if any(LA.eigvals(H) >= 1):
        raise ValueError("Computation does not converge.")
    
    current_x = np.zeros(H.shape[1])
    for step in range(1, limit):
        next_x = H @ current_x + g

        if stop(current_x, next_x):
            return next_x, step
        
        current_x = next_x
    return next_x, step


def solve(A: np.ndarray, b: np.ndarray, accuracy: float, method: SolvingMethod, limit=10000) -> tuple[np.ndarray, int]:
    H, g = convert_system(A, b)
    norm_H = LA.norm(H)

    match method:
        case SolvingMethod.SIMPLE_ITERATION:
            coefficient = norm_H / (1 - norm_H) if norm_H < 1 else 1
        case SolvingMethod.SEIDEL:
            H_l, H_r, E = np.tril(H, -1), np.triu(H), np.identity(H.shape[0])
            H, g = LA.inv(E - H_l) @ H_r, LA.inv(E - H_l) @ g
            coefficient = LA.norm(H_r) / (1 - norm_H) if norm_H < 1 else 1

    stop = lambda current_x, next_x: coefficient * LA.norm(next_x - current_x) <= accuracy
    return iterate(H, g, stop, limit)

In [3]:
methods = {
    'метод простой итерации': SolvingMethod.SIMPLE_ITERATION,
    'метод Зейделя': SolvingMethod.SEIDEL
}

columns = ['точность', 'метод', 'количество итераций', 'погрешность']

format = {columns[0]: '{:.0e}', 
          columns[3]: '{:.6e}'}

style = [{'selector': 'th', 'props': [('max-width', '100px')]}]

def results_df(matrix: np.ndarray, powers=range(2, 9)) -> pd.DataFrame:
    exact_x = np.ones(matrix.shape[0])
    b = matrix @ exact_x

    data = []
    accuracy = [10 ** -p for p in powers]
    for a in accuracy:
        for m_name, m in methods.items():
            x, steps = solve(matrix, b, a, m)
            data.append([a, m_name, steps, LA.norm(exact_x - x)])
    return pd.DataFrame(data, columns=columns)

### Симметричная с диагональным преобладанием разреженная матрица большого порядка

In [4]:
def generate(size: int, gamma=0.3, max_rand=15) -> np.ndarray:
    limit = size ** (1 + gamma)
    matrix = np.zeros((size, size))

    triu = list(zip(*np.triu_indices_from(matrix, k=1)))
    np.random.shuffle(triu)
    indices_to_fill = triu[:math.floor((limit - size) / 2)]

    for i, j in indices_to_fill:
        matrix[i, j] = (-1) ** np.random.randint(2) * np.random.uniform(-max_rand, 0)
        matrix[j, i] = matrix[i, j]

    diagonal = ((-1) ** np.random.randint(0, 2, size) * 
                np.max([np.sum(abs(matrix), 1), 
                        -np.random.uniform(-max_rand, 0, size)], 0))
    np.fill_diagonal(matrix, diagonal)
    return matrix

In [5]:
for n in range(200, 501, 100):
    print(f"n = {n}")
    matrix = generate(n)
    display(results_df(matrix).style.format(format).set_table_styles(style).hide())

n = 200


точность,метод,количество итераций,погрешность
1e-02,метод простой итерации,37,1.230641e-02
1e-02,метод Зейделя,17,2.855894e-02
1e-03,метод простой итерации,55,1.174922e-03
1e-03,метод Зейделя,26,2.725345e-03
1e-04,метод простой итерации,73,1.121840e-04
1e-04,метод Зейделя,35,2.601341e-04
1e-05,метод простой итерации,90,1.022575e-05
1e-05,метод Зейделя,43,3.223554e-05
1e-06,метод простой итерации,107,1.327754e-06
1e-06,метод Зейделя,52,3.076914e-06


n = 300


точность,метод,количество итераций,погрешность
1e-02,метод простой итерации,24,4.730285e-03
1e-02,метод Зейделя,13,2.851026e-03
1e-03,метод простой итерации,32,6.259612e-04
1e-03,метод Зейделя,17,3.747053e-04
1e-04,метод простой итерации,42,5.001127e-05
1e-04,метод Зейделя,22,2.987092e-05
1e-05,метод простой итерации,50,6.623171e-06
1e-05,метод Зейделя,27,2.383847e-06
1e-06,метод простой итерации,60,5.291107e-07
1e-06,метод Зейделя,31,3.155110e-07


n = 400


точность,метод,количество итераций,погрешность
1e-02,метод простой итерации,16,1.006996e-02
1e-02,метод Зейделя,10,6.944557e-03
1e-03,метод простой итерации,22,1.257253e-03
1e-03,метод Зейделя,13,9.395929e-04
1e-04,метод простой итерации,28,1.673988e-04
1e-04,метод Зейделя,17,6.570853e-05
1e-05,метод простой итерации,35,1.664524e-05
1e-05,метод Зейделя,20,8.956322e-06
1e-06,метод простой итерации,41,2.246335e-06
1e-06,метод Зейделя,24,6.293817e-07


n = 500


точность,метод,количество итераций,погрешность
1e-02,метод простой итерации,18,3.733847e-03
1e-02,метод Зейделя,10,2.195407e-03
1e-03,метод простой итерации,23,5.672043e-04
1e-03,метод Зейделя,13,2.107457e-04
1e-04,метод простой итерации,29,5.608831e-05
1e-04,метод Зейделя,16,2.138723e-05
1e-05,метод простой итерации,35,5.736772e-06
1e-05,метод Зейделя,19,2.259288e-06
1e-06,метод простой итерации,41,6.018499e-07
1e-06,метод Зейделя,22,2.419282e-07
